# Installation of c3py

## Step 1: install dependencies

In [1]:
!cd /work/
!apt update
!apt install -y cmake
!sudo apt-get install -y libatlas-base-dev
!sudo apt-get install -y gfortran
!sudo apt-get install -y libblas-dev liblapack-dev
!sudo apt install -y swig
!pip install pathlib
#!git clone https://github.com/goroda/Compressed-Continuous-Computation.git c3

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian-security buster/updates InRelease
Hit:3 http://deb.debian.org/debian buster-updates InRelease



54 packages can be upgraded. Run 'apt list --upgradable' to see them.



cmake is already the newest version (3.13.4-1).
0 upgraded, 0 newly installed, 0 to remove and 54 not upgraded.



libatlas-base-dev is already the newest version (3.10.3-8).
0 upgraded, 0 newly installed, 0 to remove and 54 not upgraded.



gfortran is already the newest version (4:8.3.0-1).
0 upgraded, 0 newly installed, 0 to remove and 54 not upgraded.



libblas-dev is already the newest version (3.8.0-2).
liblapack-dev is already the newest version (3.8.0-2).
0 upgraded, 0 newly installed, 0 to remove and 54 not upgraded.



swig is already the newest version (3.0.12-2).
0 upgraded, 0 newly installed, 0 to remove and 54 not upgraded.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' comma

## Step 2: modify `c3py.py` file

copy the following code:

``def get_uni_func_nparam(self, dim, row, col):
    return c3.function_train_func_get_nparams(self.ft, dim, row, col)``
    
``def get_nparam(self):
    return c3.function_train_get_nparams(self.ft)``

``def get_deg_mode(self):
    ranks = self.get_ranks()
    dim = len(ranks) - 1
    myN = np.zeros(dim) - 1
    for ii in range(dim):
        for row in range(ranks[ii]):
            for col in range(ranks[ii+1]):
                nparamuni = self.get_uni_func_nparam(ii,row,col)
                if myN[ii] < nparamuni:
                    myN[ii] = nparamuni
    return myN``

Paste them at the ``FunctionTrain`` class of the file ``c3/wrappers/python/c3py/c3py.py``.

## Setp 3: build and install the package

In [2]:
%cd /work/c3
!python setup.py build
!python setup.py install
import sys
sys.path.insert(1, '/root/venv/lib/python3.9/site-packages/c3py-0.0.7-py3.9-linux-x86_64.egg')

/work/c3
Add Extension  _c3_notsure
running build
running build_py
running build_ext
Creating Tempory File
Temporary file =  build/temp.linux-x86_64-3.9
cmake /work/c3 -DCMAKE_LIBRARY_OUTPUT_DIRECTORY=/work/c3/build/lib.linux-x86_64-3.9 -DCMAKE_BUILD_TYPE=Release -DBUILD_TESTS=OFF -DBUILD_EXAMPLES=OFF -DBUILD_PROFILING=OFF -DBUILD_BENCHMARKS=OFF -DBUILD_UTILITIES=OFF
-- A library with LAPACK API found.
-- Configuring done
-- Generating done
-- Build files have been written to: /work/c3/build/temp.linux-x86_64-3.9
cmake --build . --config Release -- -j4
[100%] Built target c3
Building Swig
['build/lib.linux-x86_64-3.9']
['$ORIGIN/']
building '_c3' extension
swigging wrappers/python/c3swig/c3.i to wrappers/python/c3swig/c3_wrap.c
swig -python -py3 -Ic3 -o wrappers/python/c3swig/c3_wrap.c wrappers/python/c3swig/c3.i
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -Ic3/lib_clinalg -Ic3/lib_interface -Ic3/lib_funcs -Ic3/lib_linalg -Ic3/lib_array -Ic3/lib_f

# Experiments

## Approximation of the 20 benchmark functions

In [3]:
# import and define test functions

import c3py
import numpy as np
import scipy.io

global numevals
numevals = 0
global selectedFunction

def func2(xx,param=None):

    if selectedFunction == 1:
        #ackley
        xx = 32.768*xx
        A = 20
        B=0.2
        C=2*np.pi
        out = -A*np.exp(-B*np.sqrt(np.mean(xx**2,axis=1))) - np.exp(np.mean(np.cos(C*xx),axis=1))+A+np.exp(1)
    if selectedFunction == 2:
        #alpine
        xx = 10*xx
        out = np.sum(np.abs(xx*np.sin(xx)+0.1*xx),axis=1)
    if selectedFunction == 3:
        #dixon
        xx = 10*xx
        out = (xx[:,0]-1)**2+ np.sum((2*xx[:,1:]**2 - xx[:,:-1])**2*(np.array(range(2,xx.shape[1]+1))),axis=1)
    if selectedFunction == 4:
        #Exponential
        out = -np.exp(-np.sum(xx**2,axis=1)/2)
    if selectedFunction == 5:
        #Grienwank
        xx = 600*xx
        out = np.sum(xx**2,axis=1)/4000 - np.prod(np.cos(xx)/np.array(range(1,xx.shape[1]+1)) ,axis=1) + 1
    if selectedFunction == 6:
        #Michalewicz
        xx = (xx+1)*np.pi/2
        m = 10
        out = -np.sum(np.sin(xx)*np.sin(xx**2*np.array(range(1,xx.shape[1]+1))/np.pi)**(2*m),axis=1)
    if selectedFunction == 7:
        #Piston
        left = np.array([30,0.005,0.002,1000,90000,290,340])
        right = np.array([60,0.02,0.01,5000,110000,296,360])
        xx = xx*(right -left)/2 + (right+left)/2

        M  = xx[:,0]
        S  = xx[:,1]
        V0 = xx[:,2]
        k  = xx[:,3]
        P0 = xx[:,4]
        Ta = xx[:,5]
        T0 = xx[:,6]

        A = P0 * S + 19.62 * M -k*V0 / S
        V = (np.sqrt(A**2 + 4*k*(P0*V0/T0)*Ta) - A)*S / (2*k)
        out = 2 * np.pi * np.sqrt(M/(k + (S**2)*(P0*V0/T0)*(Ta/(V**2))))

    if selectedFunction == 8:
        #Qing
        xx = (xx+1)*250
        out = np.sum((xx**2 - (np.array(range(1,xx.shape[1]+1))))**2,axis=1)
    if selectedFunction == 9:
        #Rastrigin
        xx = xx*5.12
        A = 10
        out = A*xx.shape[1] + np.sum(xx**2 - A*np.cos(2*np.pi*xx),axis=1)
    if selectedFunction == 10:
        #Rosenbrock
        xx = xx*2.048
        out = np.sum(100*(xx[:,1:]-xx[:,:-1]**2)**2+(1-xx[:,:-1])**2,axis=1)
    if selectedFunction == 11:
        #Schaffer
        xx = xx*100
        temp = xx[:,1:]**2+xx[:,:-1]**2
        out = np.sum( 0.5+(np.sin(np.sqrt(temp))**2-0.5)/(1+0.001*temp)**2,axis=1)
    if selectedFunction == 12:
        #Schwefel
        xx = xx*500
        out = 418.9829*xx.shape[1] - np.sum(xx*np.sin(np.sqrt(np.abs(xx))),axis=1)
    if selectedFunction == 13:
        #borehole
        left = np.array([0.05,100,63070,990,63.1,700,1120,9855])
        right = np.array([0.15,50000,115600,1110,116,820,1680,12045])
        xx = xx*(right -left)/2 + (right+left)/2

        rw = xx[:,0]
        r  = xx[:,1]
        Tu = xx[:,2]
        Hu = xx[:,3]
        Tl = xx[:,4]
        Hl = xx[:,5]
        L  = xx[:,6]
        Kw = xx[:,7]

        frac1 = 2 * np.pi * Tu * (Hu-Hl)

        frac2a = 2*L*Tu / (np.log(r/rw)*rw**2*Kw)
        frac2b = Tu / Tl
        frac2 = np.log(r/rw) * (1+frac2a+frac2b)

        out = frac1 / frac2
    if selectedFunction == 14:
        #otlcircuit
        left = np.array([50,25,0.5,1.2,0.25,50])
        right = np.array([150,70,3,2.5,1.2,300])
        xx = xx*(right -left)/2 + (right +left)/2

        Rb1  = xx[:,0]
        Rb2  = xx[:,1]
        Rf   = xx[:,2]
        Rc1  = xx[:,3]
        Rc2  = xx[:,4]
        beta = xx[:,5]

        Vb1 = 12*Rb2 / (Rb1+Rb2)
        term1a = (Vb1+0.74) * beta * (Rc2+9)
        term1b = beta*(Rc2+9) + Rf
        term1 = term1a / term1b

        term2a = 11.35 * Rf
        term2b = beta*(Rc2+9) + Rf
        term2 = term2a / term2b

        term3a = 0.74 * Rf * beta * (Rc2+9)
        term3b = (beta*(Rc2+9)+Rf) * Rc1
        term3 = term3a / term3b

        out = term1 + term2 + term3
    if selectedFunction == 15:
        #robot
        theta = (xx[:,:4] + 1)*np.pi
        L     = (xx[:,4:] + 1)/2

        sumu = 0
        sumv = 0
        for ii in range(4):
            Li = L[:,ii]
            sumtheta = 0
            for jj in range(ii+1): 
                thetai = theta[:,jj]
                sumtheta = sumtheta + thetai
            sumu = sumu + Li*np.cos(sumtheta)
            sumv = sumv + Li*np.sin(sumtheta)

        u = sumu
        v = sumv

        out = (u*u + v*v)**(0.5)
    if selectedFunction == 16:
        #wingweight
        left = np.array([150,220,6,-10,16,0.5,0.08,2.5,1700,0.025])
        right = np.array([200,300,10,10,45,1,0.18,6,2500,0.08])
        
        xx = xx*(right -left)/2 + (right +left)/2
        

        Sw      = xx[:,0]
        Wfw     = xx[:,1]
        A       = xx[:,2]
        LamCaps = xx[:,3] * (np.pi/180)
        q       = xx[:,4]
        lam     = xx[:,5]
        tc      = xx[:,6]
        Nz      = xx[:,7]
        Wdg     = xx[:,8]
        Wp      = xx[:,9]

        fact1 = 0.036 * Sw**0.758 * Wfw**0.0035
        fact2 = (A / ((np.cos(LamCaps))**2))**0.6
        fact3 = q**0.006 * lam**0.04
        fact4 = (100*tc / np.cos(LamCaps))**(-0.3)
        fact5 = (Nz*Wdg)**0.49

        term1 = Sw * Wp

        out = fact1*fact2*fact3*fact4*fact5 + term1

    if selectedFunction == 17:
        #fried
        xx = (xx + 1)/2
        out = 10 * np.sin(np.pi*xx[:,0]*xx[:,1]) + 20 * (xx[:,2]-0.5)**2 + 10*xx[:,3] + 5*xx[:,4]
    if selectedFunction == 18:
        #grlee09
        xx = (xx+1)/2
        out = np.exp(np.sin((0.9*(xx[:,0]+0.48))**10)) + xx[:,1]*xx[:,2] + xx[:,3]
    if selectedFunction == 19:
        #detpep108d
        xx = (xx+1)/2
        out = 4*(xx[:,0]-2+8*xx[:,1]-8*xx[:,1]**2)**2+(3-4*xx[:,1])**2+\
                16*np.sqrt(xx[:,2]+1)*(2*xx[:,2]-1)**2 + 4*np.log(1+np.sum(xx[:,2:4],axis=1))+\
                5*np.log(1+np.sum(xx[:,2:5],axis=1))+6*np.log(1+np.sum(xx[:,2:6],axis=1))+\
                7*np.log(1+np.sum(xx[:,2:7],axis=1))+8*np.log(1+np.sum(xx[:,2:],axis=1))
    if selectedFunction == 20:
        #detpep10exp
        xx = (xx+1)/2
        out = 100*(np.exp(-2/xx[:,0]**1.75)+np.exp(-2/xx[:,1]**1.5)+np.exp(-2/xx[:,2]**1.25))


    global numevals

    numevals = numevals + len(out)

    return out

In [4]:
# do approximation

lb = -1                
ub = 1
cross_tol=1e-10
round_tol=1e-10
approx_tol=1e-10
verbose=0
adapt=0

init_rank_list = [16,2,3,1,3,2,24,2,2,3,20,2,2,5,20,2,4,2,7,2]
dlist = 12*[7] + [8,6,8,10,5,6,8,3]
funcname = ['ackley','alpine','dixon','exponential','grienwank',
            'michalewicz','piston','qing','rastrigin','rosenbrock',
            'schaffer','schwefel','borehole','otlcircuit','robot',
            'wingweight','fried','grlee09','detpep108d','detpep10exp']

testPoints = scipy.io.loadmat('/work/testPoints.mat')
x_all = testPoints['x_all']
dlist_all = np.squeeze(testPoints['dlist_all'])

tabledata = np.zeros([20,5])

print("|   function  |   error  |   eval  |   dof  | degree | TTrank |")

for ii in range(20):

    numevals = 0
    selectedFunction = ii+1

    init_rank = init_rank_list[ii]
    d = dlist[ii]
    d_index = np.where(dlist_all==d)[0][0]
    x = x_all[d_index][0]
    y = func2(x)
    
    ftc = c3py.FunctionTrain(d)
    for jj in range(d):
        ftc.set_dim_opts(jj,"legendre",lb,ub,tol=approx_tol,coeff_check=4,nparam=14)

    ftc.build_approximation(func2,None,init_rank,verbose,adapt,\
    cross_tol=cross_tol,round_tol=round_tol, maxiter=1,maxrank=25,kickrank=5)

    yhat = ftc.eval(x)
    
    # error, evals, dof, degree, ranks
    tabledata[ii,0] = np.linalg.norm(y-yhat)/np.linalg.norm(y)
    tabledata[ii,1] = numevals
    tabledata[ii,2] = ftc.get_nparam()
    tabledata[ii,3] = ftc.get_deg_mode().max()
    tabledata[ii,4] = ftc.get_ranks().max()

    print("| %11s | %8.2e | %7.i | %6.i |   %3.i  |   %2.i   |"
        %(funcname[ii],tabledata[ii,0],tabledata[ii,1],tabledata[ii,2],tabledata[ii,3],tabledata[ii,4]))
    np.save('/work/data/testFT_20BenchFuncs.npy', tabledata)

|   function  |   error  |   eval  |   dof  | degree | TTrank |
|      ackley | 1.81e-02 | 2232528 | 137760 |   105  |   16   |
|      alpine | 6.43e-03 |   50656 |   2520 |   105  |    2   |
|       dixon | 2.89e-14 |   13752 |    435 |    21  |    3   |
| exponential | 1.71e-14 |   10518 |    133 |    19  |    1   |
|   grienwank | 3.52e-06 |   51466 |   4459 |   105  |    3   |
| michalewicz | 1.45e-01 |   48745 |   2443 |   105  |    2   |
|      piston | 3.85e-05 |  251760 |  66080 |    35  |   24   |
|        qing | 2.86e-14 |   11776 |    136 |    21  |    2   |
|   rastrigin | 1.86e-10 |   22288 |   1342 |    63  |    2   |
|  rosenbrock | 9.43e-14 |   11530 |    633 |    14  |    3   |
|    schaffer | 1.22e-01 | 3465760 | 214200 |   105  |   20   |
|    schwefel | 5.45e-04 |   50656 |   2496 |   104  |    2   |
|    borehole | 2.08e-03 |   14346 |    577 |    70  |    2   |
|  otlcircuit | 4.07e-08 |   15674 |   1782 |    28  |    5   |
|       robot | 3.85e-01 | 2018017 | 228

## Integration of rank-two sine functions

In [5]:
# import and define test functions
import c3py
import numpy as np

global numevals
numevals = 0

def func2(xx,param=None):
    out = np.sin(np.sum(xx+1,axis=1)/2)

    global numevals
    numevals = numevals + len(out)

    return out

In [6]:
# do approximation
import numpy as np

lb = -1
ub = 1
init_rank = 2
dlist = np.linspace(10,450,10)
tollist = np.logspace(-13, -1, 10)
tollist = tollist[[0,4,9]]
verbose=0
adapt=0
tabledata = np.zeros([len(dlist),len(tollist),4])

print("|  d  |   tol   |   error  |  eval  |  dof  | degree |")
for ii in range(len(dlist)):
    d = int(dlist[ii])
    I_true = np.imag( ((np.exp(1j)-1)/1j)**d )*2**d
    for jj in range(len(tollist)):
        tol = tollist[jj]
        numevals = 0
        
        ftc = c3py.FunctionTrain(d)
        for kk in range(d):
            ftc.set_dim_opts(kk,"legendre",lb,ub,tol=tol,coeff_check=2,nparam=6)

        
        ftc.build_approximation(func2,None,init_rank,verbose,adapt,cross_tol=tol,round_tol=tol, maxiter=1)
        
        I_est = ftc.integrate()

        error = np.abs(I_true-I_est)/np.abs(I_true)
        evals = numevals
        dof = ftc.get_nparam()

        tabledata[ii,jj,0] = error
        tabledata[ii,jj,1] = evals
        tabledata[ii,jj,2] = dof
        tabledata[ii,jj,3] = ftc.get_deg_mode().max()
        print("| %3.i | %7.1e | %8.2e | %6.i | %5.i |   %2.i   |"
            %(d,tol,tabledata[ii,jj,0],tabledata[ii,jj,1],tabledata[ii,jj,2],tabledata[ii,jj,3]))
        np.save('/work/data/testFT_SineIntegration.npy', tabledata)

|  d  |   tol   |   error  |  eval  |  dof  | degree |
|  10 | 1.0e-13 | 2.17e-14 |   1512 |   417 |   12   |
|  10 | 2.2e-08 | 2.17e-14 |   1512 |   417 |   12   |
|  10 | 1.0e-01 | 8.89e-06 |    504 |   216 |    6   |
|  58 | 1.0e-13 | 1.54e-13 |   9618 |  2625 |   12   |
|  58 | 2.2e-08 | 1.54e-13 |   9576 |  2625 |   12   |
|  58 | 1.0e-01 | 1.14e-04 |   3192 |  1368 |    6   |
| 107 | 1.0e-13 | 1.49e-12 |  17934 |  4879 |   12   |
| 107 | 2.2e-08 | 1.49e-12 |  17808 |  4879 |   12   |
| 107 | 1.0e-01 | 8.67e-04 |   5936 |  2544 |    6   |
| 156 | 1.0e-13 | 1.08e-12 |  26292 |  7133 |   12   |
| 156 | 2.2e-08 | 8.29e-13 |  26040 |  7133 |   12   |
| 156 | 1.0e-01 | 3.73e-05 |   8680 |  3720 |    6   |
| 205 | 1.0e-13 | 1.13e-12 |  34692 |  9387 |   12   |
| 205 | 2.2e-08 | 1.13e-12 |  34272 |  9387 |   12   |
| 205 | 1.0e-01 | 2.14e-04 |  11424 |  4896 |    6   |
| 254 | 1.0e-13 | 1.16e-12 |  42945 | 11641 |   12   |
| 254 | 2.2e-08 | 1.16e-12 |  42504 | 11641 |   12   |
| 254 | 1.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c1887c9a-cd78-4951-a9a4-f35cb582c2ea' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>